# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846813336917                   -0.70    4.5
  2   -7.852314567604       -2.26       -1.53    1.0
  3   -7.852616546479       -3.52       -2.55    1.5
  4   -7.852645904173       -4.53       -2.89    2.2
  5   -7.852646511896       -6.22       -3.21    1.2
  6   -7.852646677361       -6.78       -4.15    1.0
  7   -7.852646686673       -8.03       -4.99    2.2
  8   -7.852646686723      -10.30       -5.41    1.2
  9   -7.852646686729      -11.26       -5.76    1.2
 10   -7.852646686730      -11.96       -6.78    1.2


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846851324295                   -0.70           4.8
  2   -7.852526116668       -2.25       -1.64   0.80    2.0
  3   -7.852635742539       -3.96       -2.73   0.80    1.0
  4   -7.852646518621       -4.97       -3.26   0.80    2.2
  5   -7.852646674766       -6.81       -4.12   0.80    1.0
  6   -7.852646686381       -7.94       -4.79   0.80    1.5
  7   -7.852646686725       -9.46       -5.61   0.80    1.8
  8   -7.852646686730      -11.30       -7.05   0.80    2.0


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.381742e+01     3.306794e+00
 * time: 0.5168359279632568
     1     1.418931e+00     1.803145e+00
 * time: 0.7834880352020264
     2    -1.477991e+00     2.030239e+00
 * time: 0.819586992263794
     3    -3.561681e+00     1.722115e+00
 * time: 0.8733608722686768
     4    -4.917868e+00     1.649345e+00
 * time: 0.9246330261230469
     5    -6.647626e+00     1.075224e+00
 * time: 0.9747450351715088
     6    -7.312081e+00     4.562381e-01
 * time: 1.0241029262542725
     7    -7.527468e+00     1.437143e-01
 * time: 1.0626678466796875
     8    -7.626323e+00     2.075301e-01
 * time: 1.0987110137939453
     9    -7.715972e+00     1.075246e-01
 * time: 1.1389729976654053
    10    -7.788151e+00     9.936673e-02
 * time: 1.179692029953003
    11    -7.825469e+00     6.481275e-02
 * time: 1.2187750339508057
    12    -7.845002e+00     5.083492e-02
 * time: 1.257817029953003
    13    -7.848651e+00     4.594093e-02
 * time: 1.2984609603881

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846828602137                   -0.70    4.8


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852645884750                   -1.64
  2   -7.852646686730       -6.10       -3.71
  3   -7.852646686730      -13.24       -7.23


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.6950609869099965e-7
|ρ_newton - ρ_scfv| = 7.679492744666548e-8
|ρ_newton - ρ_dm|   = 1.4587853457304449e-9
